# Other Models

I'd like to try several other simple models included in sklearn.  SVM, Random Forest, and Hidden Markov seem like good places to start.  I will make copies of `lr_train.py` for each model.  In principle I could use a different model for each of root, quality, add, and inversion.

## Setup

In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import sys, os
from sklearn.model_selection import train_test_split
import sklearn
import re
import pickle

if 'chord_loader' in sys.modules:
    del sys.modules['chord_loader']
sys.path.append('.')
import chord_loader

In [2]:
#copy code from lr_train so we can get to the point where these models can be tested. 
#Use 10% data for this purpose
source = 'partial_lr'
source_dir = 'Data/processed/'

#get information from processed data directory
data_info = pd.read_csv(source_dir + 'directory.csv')
curr_data_info = data_info.loc[data_info['filepath']==source,:]
if curr_data_info.shape[0] < 1:
    print('Source not found in directory')
    sys.exit(1)
curr_data_info = curr_data_info.iloc[-1,:]

#load data
features_train = np.load(f'{source_dir}{source}_ftrain.npy')
labels_train = np.load(f'{source_dir}{source}_ltrain.npy')
features_valid = np.load(f'{source_dir}{source}_fvalid.npy')
labels_valid = np.load(f'{source_dir}{source}_lvalid.npy')
features_test = np.load(f'{source_dir}{source}_ftest.npy')
labels_test = np.load(f'{source_dir}{source}_ltest.npy')
if curr_data_info['standard']:
    standard_features_train = np.load(f'{source_dir}{source}_fstrain.npy')
    standard_labels_train = np.load(f'{source_dir}{source}_lstrain.npy')
    standard_features_valid = np.load(f'{source_dir}{source}_fsvalid.npy')
    standard_labels_valid = np.load(f'{source_dir}{source}_lsvalid.npy')
    standard_features_test = np.load(f'{source_dir}{source}_fstest.npy')
    standard_labels_test = np.load(f'{source_dir}{source}_lstest.npy')


## SVM